In [ ]:
import pandas as pd
import numpy as np
import requests
from PIL import Image, ImageSequence
from io import BytesIO
import matplotlib.pyplot as plt
from IPython.display import Image as IPyImage, display
import imageio.v2 as imageio
from tqdm import tqdm
from IPython.display import HTML
from concurrent.futures import ThreadPoolExecutor, as_completed
import imageio

In [2]:
data = pd.read_csv("/Users/ishananand/Desktop/Text-To-Video-Generation/data/tgif-v1.0.tsv", sep='\t', header=None, names=['url', 'caption'])
data.head()

,url,caption
0,https://38.media.tumblr.com/9f6c25cc350f12aa74...,"a man is glaring, and someone with sunglasses ..."
1,https://38.media.tumblr.com/9ead028ef62004ef6a...,a cat tries to catch a mouse on a tablet
2,https://38.media.tumblr.com/9f43dc410be85b1159...,a man dressed in red is dancing.
3,https://38.media.tumblr.com/9f659499c8754e40cf...,an animal comes close to another in the jungle
4,https://38.media.tumblr.com/9ed1c99afa7d714118...,a man in a hat adjusts his tie and makes a wei...


In [3]:
data['frames'] = 0

In [4]:
data.head()

,url,caption,frames
0,https://38.media.tumblr.com/9f6c25cc350f12aa74...,"a man is glaring, and someone with sunglasses ...",0
1,https://38.media.tumblr.com/9ead028ef62004ef6a...,a cat tries to catch a mouse on a tablet,0
2,https://38.media.tumblr.com/9f43dc410be85b1159...,a man dressed in red is dancing.,0
3,https://38.media.tumblr.com/9f659499c8754e40cf...,an animal comes close to another in the jungle,0
4,https://38.media.tumblr.com/9ed1c99afa7d714118...,a man in a hat adjusts his tie and makes a wei...,0


In [5]:
data.shape

(125782, 3)

In [6]:
rl = data['url'][0]
resp = requests.get(rl)
img = Image.open(BytesIO(resp.content))
HTML(f'<img src="{rl}" />')


In [ ]:
invalid_urls = []

def process_url(i, url):
    try:
        resp = requests.get(url, timeout=10)
        if resp.status_code != 200:
            return i, None
        gif = imageio.mimread(BytesIO(resp.content))
        return i, len(gif)
    except Exception:
        return i, None

results = []
with ThreadPoolExecutor(max_workers=80) as executor:
    futures = {executor.submit(process_url, i, url): i for i, url in enumerate(data['url'])}
    for future in tqdm(as_completed(futures), total=len(futures), desc="Processing URLs"):
        i, frame_count = future.result()
        if frame_count is None:
            invalid_urls.append(i)
        else:
            data.loc[i, "frames"] = frame_count


Processing URLs: 100%|██████████| 125782/125782 [46:27<00:00, 45.13it/s] 


In [ ]:
data.drop(index=invalid_urls, inplace=True)
data.to_csv("/Users/ishananand/Desktop/Text-To-Video-Generation/data/modified_tgif.csv", index=False, columns = ['url', 'caption', 'frames'])

In [33]:
len(invalid_urls)

99

In [29]:

resp = requests.get(df['url'][12946], timeout=10)
gif = imageio.mimread(BytesIO(resp.content))
len(gif)

30

In [30]:
df = pd.read_csv("/Users/ishananand/Desktop/Text-To-Video-Generation/data/modified_tgif.csv")
HTML(f'<img src="{df['url'][1]}" />')


In [32]:
data[data['frames'] <= 40].shape

(80387, 3)

In [17]:
data['frames'].max()
data[data['frames'] == 572]['url']

12949    https://38.media.tumblr.com/61e04161c76a2f3bd1...
Name: url, dtype: object

In [ ]:
HTML(f'<img src="{data[data['frames'] == 572]['url']}" />')

In [109]:
gif = imageio.mimread(BytesIO(resp.content))
print(f"Loaded {len(gif)} frames from GIF")

Loaded 18 frames from GIF


In [100]:
img = np.array(img)
img.shape

(243, 500, 4)

In [96]:
img

array([[  7,   5,   5, 255],
       [  0,   0,   0, 255],
       [  3,   3,   2, 255],
       ...,
       [ 58,  53,  49, 255],
       [ 53,  46,  45, 255],
       [ 77,  72,  67, 255]], shape=(500, 4), dtype=uint8)